In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from skimage import io
import os
import gc
import pickle


from skimage.filters import gaussian
from skimage.morphology import remove_small_objects
from scipy.ndimage import center_of_mass
from stardist.models import StarDist2D
from csbdeep.utils import normalize
from skimage.segmentation import clear_border

In [2]:
dataDir='/media/xinyi/T7 Shield/neuro'
phenoLabels=['Control','AD','IPD','FTLD-TDPC','PSP']
tif3dpath='TIFS/3DOrig'
stains=['GFAP','MAP2','LMNB','DAPI']
proj2dpath='TIFS/2DMaxProj'

### 2D projection and generate missing masks

In [ ]:
#generate 2d max project
for p in phenoLabels:
    print(p)
    if not os.path.exists(os.path.join(dataDir,p,proj2dpath)):
        os.mkdir(os.path.join(dataDir,p,proj2dpath))
    for s in os.listdir(os.path.join(dataDir,p,tif3dpath)):
        print(s)
        #load 2d imgs
        img3d=io.imread(os.path.join(dataDir,p,tif3dpath,s))
        for stainIdx in range(len(stains)):
            if not os.path.exists(os.path.join(dataDir,p,proj2dpath,stains[stainIdx])):
                os.mkdir(os.path.join(dataDir,p,proj2dpath,stains[stainIdx]))
            io.imsave(os.path.join(dataDir,p,proj2dpath,stains[stainIdx],s),np.max(img3d[:,:,:,stainIdx],axis=0))

In [6]:
model_stardist = StarDist2D.from_pretrained('2D_versatile_fluo')

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [ ]:
#generate stardist mask for all
segPath_new='TIFS/segmentations_new'
for p in phenoLabels:
    print(p)
    if not os.path.exists(os.path.join(dataDir,p,segPath_new)):
        os.mkdir(os.path.join(dataDir,p,segPath_new))
    for s in os.listdir(os.path.join(dataDir,p,tif3dpath)):
        if os.path.exists(os.path.join(dataDir,p,segPath_new,s)):
            continue
        print(s)
        #load 2d dapi
        img2d=io.imread(os.path.join(dataDir,p,proj2dpath,'DAPI',s))
        labeled_nuclei, linfo = model_stardist.predict_instances(gaussian(normalize(img2d),5),n_tiles=(16,16))
        io.imsave(os.path.join(dataDir,p,segPath_new,s),labeled_nuclei)

### filter by size, clear border, rescale intensity, save stains, save patient ID, save coordinates

add control batches

channel order: dapi,gfap,map2,lmnb

In [4]:
newbatchID=['P10-20-1','P10-20-2','P16-19-1','P16-19-2','P33-18-1','P33-18-2','P38-19-1','P38-19-2','P38-19-3','P49-18-1','P49-18-2','P49-18-3','P85-19-1','P85-19-2','P89-19-1','P89-19-2']

In [ ]:
segPath_new='TIFS/segmentations_new'
imgSize=128
radius=int(imgSize/2)
allImgs=None
allCoord=None
allImgNames=None
allPatientID=None
allCat=None

minNucSize=300
laminThreshold=0.3 #by area
minmaxScale=(10,99.99)
for p in phenoLabels:
    print(p)
    
    for s in os.listdir(os.path.join(dataDir,p,tif3dpath)):
        print(s)
        #load 2d dapi
        img2d=io.imread(os.path.join(dataDir,p,proj2dpath,'DAPI',s))
        img2d[img2d<np.percentile(img2d,minmaxScale[0])]=np.percentile(img2d,minmaxScale[0])
        img2d=(img2d-np.min(img2d))/(np.max(img2d)-np.min(img2d))
        #load 2d gfap
        img2d_gfap=io.imread(os.path.join(dataDir,p,proj2dpath,'GFAP',s))
        img2d_gfap[img2d_gfap<np.percentile(img2d_gfap,minmaxScale[0])]=np.percentile(img2d_gfap,minmaxScale[0])
        img2d_gfap=(img2d_gfap-np.min(img2d_gfap))/(np.max(img2d_gfap)-np.min(img2d_gfap))
        #load 2d map2
        img2d_map2=io.imread(os.path.join(dataDir,p,proj2dpath,'MAP2',s))
        img2d_map2[img2d_map2<np.percentile(img2d_map2,minmaxScale[0])]=np.percentile(img2d_map2,minmaxScale[0])
        img2d_map2=(img2d_map2-np.min(img2d_map2))/(np.max(img2d_map2)-np.min(img2d_map2))
        #load 2d lmnb
        img2d_lmnb=io.imread(os.path.join(dataDir,p,proj2dpath,'LMNB',s))
        img2d_lmnb[img2d_lmnb<np.percentile(img2d_lmnb,minmaxScale[0])]=np.percentile(img2d_lmnb,minmaxScale[0])
        img2d_lmnb=(img2d_lmnb-np.min(img2d_lmnb))/(np.max(img2d_lmnb)-np.min(img2d_lmnb))
        
        imgSeg=io.imread(os.path.join(dataDir,p,segPath_new,s))
        imgSeg=clear_border(imgSeg)
        
        currImgs=np.zeros((np.unique(imgSeg).size-1,4,imgSize,imgSize))
        currCoord=np.zeros((np.unique(imgSeg).size-1,2))
        imgCount=0
        for idx in np.unique(imgSeg):
            if idx==0:
                continue
            current_nuc=imgSeg==idx
            
            x,y = center_of_mass(current_nuc)
            x=int(x)
            y=int(y)
            if x-radius<0 or x+radius>(img2d.shape[0]) or y-radius<0 or y+radius>(img2d.shape[1]):
                continue
            
            if np.sum(current_nuc)<minNucSize:
                lmnbPix=img2d_lmnb[current_nuc]>0
                if np.sum(lmnbPix)/np.sum(current_nuc)<laminThreshold:
                    continue
            currImgs_0=np.copy(img2d[x-radius:x+radius,y-radius:y+radius])
            currImgs_0_seg=imgSeg[x-radius:x+radius,y-radius:y+radius]
            currImgs_0[currImgs_0_seg!=idx]=0
            currImgs_3=np.copy(img2d_lmnb[x-radius:x+radius,y-radius:y+radius])
            currImgs_3[currImgs_0_seg!=idx]=0
            currImgs[imgCount,0]=currImgs_0
            currImgs[imgCount,1]=img2d_gfap[x-radius:x+radius,y-radius:y+radius]
            currImgs[imgCount,2]=img2d_map2[x-radius:x+radius,y-radius:y+radius]
            currImgs[imgCount,3]=currImgs_3
            currCoord[imgCount]=np.array([x,y])
            
            imgCount+=1
        currImgs=currImgs[:imgCount]
        currCoord=currCoord[:imgCount]
        sname=s.split('.')[0]
        currImgName=np.repeat(sname,imgCount)
        currPatientID=np.repeat(sname.split('-')[0]+'-'+sname.split('-')[1],imgCount)
        currCat=np.repeat(p,imgCount)
        
        if allImgs is None:
            allImgs=currImgs
            allImgNames=currImgName
            allCoord=currCoord
            allPatientID=currPatientID
            allCat=currCat
        else:
            allImgs=np.concatenate((allImgs,currImgs),axis=0)
            allImgNames=np.concatenate((allImgNames,currImgName))
            allCoord=np.concatenate((allCoord,currCoord),axis=0)
            allPatientID=np.concatenate((allPatientID,currPatientID))
            allCat=np.concatenate((allCat,currCat))

In [63]:
savedir_processed=os.path.join(dataDir,'processed')
with open(os.path.join(savedir_processed,'allImgs_noMaxPerc_minmax_segNuc'), 'wb') as output:
    pickle.dump(allImgs, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allImgNames'), 'wb') as output:
    pickle.dump(allImgNames, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allCoord'), 'wb') as output:
    pickle.dump(allCoord, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allPatientID'), 'wb') as output:
    pickle.dump(allPatientID, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(savedir_processed,'allCat'), 'wb') as output:
    pickle.dump(allCat, output, pickle.HIGHEST_PROTOCOL)

In [8]:
allImgs.shape

(97530, 4, 128, 128)

### figure 1 crops

In [17]:
p='Control'
s='P21-17.tif'
img2d_dapi=io.imread(os.path.join(dataDir,p,proj2dpath,'DAPI',s))
#load 2d gfap
img2d_gfap=io.imread(os.path.join(dataDir,p,proj2dpath,'GFAP',s))
img2d_gfap=(img2d_gfap-np.min(img2d_gfap))/(np.max(img2d_gfap)-np.min(img2d_gfap))
#load 2d map2
img2d_map2=io.imread(os.path.join(dataDir,p,proj2dpath,'MAP2',s))
#load 2d lmnb
img2d_lmnb=io.imread(os.path.join(dataDir,p,proj2dpath,'LMNB',s))


savepath='/media/xinyi/T7 Shield/neuro/processed'
# plt.figure(figsize = (20,20))
plt.imsave(os.path.join(savepath,'fig1dapi.tiff'),img2d_dapi[1000:2500,1000:4000],vmin=np.percentile(img2d_dapi,10),vmax=np.percentile(img2d_dapi,99.99))

In [18]:
plt.imsave(os.path.join(savepath,'fig1gfap.tiff'),img2d_gfap[1000:2500,1000:4000],vmin=np.percentile(img2d_gfap,10),vmax=np.percentile(img2d_gfap,99.99))

In [19]:
plt.imsave(os.path.join(savepath,'fig1map2.tiff'),img2d_map2[1000:2500,1000:4000],vmin=np.percentile(img2d_map2,10),vmax=np.percentile(img2d_map2,99.99))

In [20]:
plt.imsave(os.path.join(savepath,'fig1lmnb.tiff'),img2d_lmnb[1000:2500,1000:4000],vmin=np.percentile(img2d_lmnb,10),vmax=np.percentile(img2d_lmnb,99.99))